## Welcome to MR simulation 
In this interactive notebook I'll guide you through using 2 powerfull github repositories and code to simulate MR signal acquisition. </br>
Credit to Eva Alonso Ortiz and Charles Pageot for *susceptibility-to-fieldmap-fft* 

In [1]:
import numpy as np
import nibabel as nib
import os
import matplotlib.pyplot as plt

## Importing, cloning and installing

In [2]:
!git clone https://github.com/shimming-toolbox/tissue-to-MRproperty.git

fatal: destination path 'tissue-to-MRproperty' already exists and is not an empty directory.


In [2]:
%cd tissue-to-MRproperty
!pip install .
%cd ..

C:\Users\User\msc_project\Image-processing-strategies\MRsim\tissue-to-MRproperty
Processing c:\users\user\msc_project\image-processing-strategies\mrsim\tissue-to-mrproperty
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tissue-to-MRproperty: filename=tissue_to_MRproperty-0.0.0-py3-none-any.whl size=29855 sha256=da875df9b4892841e52c736b72238e49db61e47bd86463523cd86ade6475c6a8
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\ce\8f\65\c868b29f9cf7cbb6272746dda3ed0e1945ec499eb343897991
Successfully built tissue-to-MRproperty
  Attempting uninstall: tissue-to-MRproperty
    Found existing installation: tissue-to-MRproperty 0.0.0
    Uninstalling tissue-to-MRproperty-0.0.0:
      Successfully uninstalled tissue-to-MRproperty-0.0.0
C:\Users\User\msc_project\Image-processing-strategies\MRsim


In [7]:
!git clone https://github.com/shimming-toolbox/susceptibility-to-fieldmap-fft.git

Cloning into 'susceptibility-to-fieldmap-fft'...


In [3]:
%cd susceptibility-to-fieldmap-fft
!pip install .
%cd ..

C:\Users\User\msc_project\Image-processing-strategies\MRsim\susceptibility-to-fieldmap-fft
Processing c:\users\user\msc_project\image-processing-strategies\mrsim\susceptibility-to-fieldmap-fft
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for susceptibility-to-fieldmap-fft: filename=susceptibility_to_fieldmap_fft-0.0.0-py3-none-any.whl size=15984 sha256=2ef42b7f6b8aae6e5613805acfefa98002f4eae67cb3567213666c69b30c94b4
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\89\fe\26\78bb69b2c55cd65a6250f1b426eed9e7c2459733c05463bafe
Successfully built susceptibility-to-fieldmap-fft
  Attempting uninstall: susceptibility-to-fieldmap-fft
    Found existing installation: susceptibility-to-fieldmap-fft 0.0.0
    Uninstalling susceptibility-to-fieldmap-fft-0.0.0:
      Successfully uninstalled susceptibility-to-fieldmap-fft-0.0.0
C:\Users\User\msc_project\Image-processing-strategies\MRsim


## Prepare for Simulation
For the simulation of MR signals we can guide ourselves with a simplified equation for simulating GRE acquisition. </br>
$$ \text{protonDensity} \cdot \sin(\text{FA}) \cdot \exp\left(-\frac{TE}{T2^*} - \text{sign} \cdot i \cdot \gamma \cdot \delta B_0 \cdot TE\right) $$

This equation requires as input the *flip angle* and the list of *echo times*, and it requires for simulation purposes:
- T2star volume
- PD volume
- delta B0
The delta B0 requires the susceptibility distribution volume. </br>

For the 3 volume requirements we use tissue to MR property repository whereas for deltaB0 we use the susceptibility to fieldmap repository.

Tissue to MR repo will automatically create 3 folders: data, output and simulation. This is for them to store any heavy information as Nifti images of processing that will be ignored by git. </br>
When you run the converter you'll find everything under *output* </br>
I recommend that you copy and paste this in your commands to your terminal because a new feature was implemented called *check pixel integrity* that checks the pixels have correct label ID before doing the conversion. This function will talk with you to correct the values telling you where the mistake pixel is located so you can open a viewer like ITK or FSLeyes and check what value it should have. This mistake arrises while adding labels and is fairly common, hence the addition of the method.

In [ ]:
!tissue_to_mr ratatouille.nii.gz -t sus -s TotalSeg_CT -v mod2 sus_volume.nii.gz

In [15]:
!tissue_to_mr ratatouille.nii.gz -t t2 -s TotalSeg_CT -v mod2 t2s_volume.nii.g

^C


In [ ]:
!tissue_to_mr ratatouille.nii.gz -t pd -s TotalSeg_CT -v mod2 pd_volume.nii.gz

One very important argument we need to calculate is the deltaB0, for which we use Susceptibility to Fieldmap fft repository. </br>
Which estimates the magnetic field perturbation that arises when an object is placed withing a magnetic field. </br>
After pip installing the repo run the command compute_fieldmap and give it the susceptibility distribution volume created before

In [34]:
!compute_fieldmap -i "data/sus_volume.nii.gz" -o "data/fieldmap.nii.gz"


Start
Susceptibility distribution loaded
Fieldmap simulated
Saving to NIfTI format
End. Runtime: 67.59 seconds


## Simulation
Now we have all the necessary Nifti files to simulate MR signal acquisition. The folder *utils* contains the necessary code for simulating with comments.

In [2]:
%cd utils
from sim_functions import optimize_measurement
%cd ..

c:\Users\User\msc_project\Image-processing-strategies\MRsim\utils
c:\Users\User\msc_project\Image-processing-strategies\MRsim


c:\Users\User\miniconda3\envs\mrsim\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# Edit the echo time (or list of echo times) and the flip angle:
TE_list = [0.008, 0.0095]#, 0.011, 0.0125, 0.014, 0.0155]
te_secs = [8, 9.5] # Using it in seconds to make it easier to run with texture on T2s values
flip_ang = 24
# The last requirement for the code is the dimensions, which can be easily acquired from the volume you work with
# The dimensions needed because we will create a 4D array, where we will store the echo time data and we want to maintain the initial 3D dimensions.
data = nib.load("data/ratatouille_crop.nii.gz")
np_data = data.get_fdata()
dimensions = np.array(np_data.shape)
dimensions

array([109, 129, 218])

In [4]:
# Loading the data 
pd= nib.load("data/pd_volume_crop.nii.gz")
pd_vol = pd.get_fdata()
t2s = nib.load("data/t2s_volume_crop.nii.gz")
t2s_vol = t2s.get_fdata()
#deltaB0 = nib.load("data/fieldmap_crop.nii.gz")
pro_deltaB0 = nib.load("output/realistic_fieldmap.nii.gz")
dB0 = pro_deltaB0.get_fdata()
fieldStrength = 3
magnitude, phase = optimize_measurement(pd_vol, t2s_vol,dimensions, dB0, flip_ang, TE_list, fieldStrength) # By default, B0 should be 3 Tesla

Starting optimize_measurement
Processing TE[0] = 0.008
Starting optimized_signal
sin:  0.4067366430758002
handedness=left
Coefficient of phase factor:  -6.420532498559999j
Finished optimized_signal
mag shape: (109, 129, 218), phase_arr shape: (109, 129, 218)
Processing TE[1] = 0.0095
Starting optimized_signal
sin:  0.4067366430758002
handedness=left
Coefficient of phase factor:  -7.624382342039999j
Finished optimized_signal
mag shape: (109, 129, 218), phase_arr shape: (109, 129, 218)
Finished optimize_measurement


In [5]:
magnitude

array([[[[45.30790962, 43.40716326],
         [45.30790962, 43.40716326],
         [45.30790962, 43.40716326],
         ...,
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[45.30790962, 43.40716326],
         [45.30790962, 43.40716326],
         [45.30790962, 43.40716326],
         ...,
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[45.30790962, 43.40716326],
         [45.30790962, 43.40716326],
         [45.30790962, 43.40716326],
         ...,
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        ...,

        [[12.70309162, 11.63025747],
         [12.70309162, 11.63025747],
         [12.70309162, 11.63025747],
         ...,
         [37.3517777 , 35.69345168],
         [37.3517777 , 35.69345168],
         [37.3517777 , 35.69345168]],

        [[12.70309162, 11.63025747

In [15]:
t2s_vol

array([[[3.500e+01, 3.500e+01, 3.500e+01, ..., 1.000e-02, 1.000e-02,
         1.000e-02],
        [3.500e+01, 3.500e+01, 3.500e+01, ..., 1.000e-02, 1.000e-02,
         1.000e-02],
        [3.500e+01, 3.500e+01, 3.500e+01, ..., 1.000e-02, 1.000e-02,
         1.000e-02],
        ...,
        [1.700e+01, 1.700e+01, 1.700e+01, ..., 3.303e+01, 3.303e+01,
         3.303e+01],
        [1.700e+01, 1.700e+01, 1.700e+01, ..., 3.303e+01, 3.303e+01,
         3.303e+01],
        [1.700e+01, 1.700e+01, 1.700e+01, ..., 3.303e+01, 3.303e+01,
         3.303e+01]],

       [[3.500e+01, 3.500e+01, 3.500e+01, ..., 1.000e-02, 1.000e-02,
         1.000e-02],
        [3.500e+01, 3.500e+01, 3.500e+01, ..., 1.000e-02, 1.000e-02,
         1.000e-02],
        [3.500e+01, 3.500e+01, 3.500e+01, ..., 1.000e-02, 1.000e-02,
         1.000e-02],
        ...,
        [1.700e+01, 1.700e+01, 1.700e+01, ..., 3.303e+01, 3.303e+01,
         3.303e+01],
        [1.700e+01, 1.700e+01, 1.700e+01, ..., 3.303e+01, 3.303e+01,
   

In [4]:
# Loading the data, Gaussian distributed volumes
g_pd= nib.load("data/gauss_pd_dist.nii.gz")
gauss_pd_vol = g_pd.get_fdata()
g_t2s = nib.load("data/gauss_t2s_dist.nii.gz")
gauss_t2s_vol = g_t2s.get_fdata()
#deltaB0 = nib.load("data/fieldmap_crop.nii.gz")
pro_deltaB0 = nib.load("output/realistic_fieldmap.nii.gz")
dB0 = pro_deltaB0.get_fdata()
fieldStrength = 3
# suppress warnings due to Overflow on exponential while iterating
#import warnings
#warnings.filterwarnings('ignore') 


In [14]:
gauss_t2s_vol[0]

array([[ 3.48625585e+01,  3.49524522e+01,  3.49688153e+01, ...,
         7.03690041e-02,  6.19318019e-02,  8.54015464e-02],
       [ 3.49597078e+01,  3.50110342e+01,  3.50136215e+01, ...,
        -4.32574542e-02, -4.96847170e-02, -1.01820534e-01],
       [ 3.49738906e+01,  3.50174988e+01,  3.50932486e+01, ...,
         2.92334431e-02, -4.68866122e-02,  2.19747050e-02],
       ...,
       [ 1.70376248e+01,  1.70430140e+01,  1.69576129e+01, ...,
         3.29806336e+01,  3.27081412e+01,  3.29513769e+01],
       [ 1.70982998e+01,  1.69079057e+01,  1.71207961e+01, ...,
         3.29677494e+01,  3.33215554e+01,  3.32167616e+01],
       [ 1.69845093e+01,  1.71207961e+01,  1.69777973e+01, ...,
         3.29805946e+01,  3.30794351e+01,  3.29922797e+01]])

In [17]:
decay = np.exp(-TE_list[0] / gauss_t2s_vol[2])

In [4]:
for te_idx, TE_val in enumerate(TE_list):
    print(f"Processing TE[{te_idx}] = {TE_val}")

Processing TE[0] = 0.008
Processing TE[1] = 0.0095


In [5]:
# For gaussian data
gauss_magnitude, gauss_phase = optimize_measurement(gauss_pd_vol, gauss_t2s_vol, dimensions, dB0, flip_ang, TE_list, fieldStrength)
# By default, B0 should be 3 Tesla

Starting optimize_measurement
Processing TE[0] = 0.008
Starting optimized_signal


c:\Users\User\msc_project\Image-processing-strategies\MRsim\utils\sim_functions.py:119: RuntimeWarning: overflow encountered in exp
  decay_gauss[i,j,k] = np.exp(-te / T2star_vol[i,j,k])


sin:  0.4067366430758002
handedness=left
Coefficient of phase factor:  -6.420532498559999j
Finished optimized_signal
mag shape: (109, 129, 218), phase_arr shape: (109, 129, 218)
Processing TE[1] = 0.0095
Starting optimized_signal
sin:  0.4067366430758002
handedness=left
Coefficient of phase factor:  -7.624382342039999j
Finished optimized_signal
mag shape: (109, 129, 218), phase_arr shape: (109, 129, 218)
Finished optimize_measurement


In [8]:
gauss_t2s_vol[10,10,10]

C:\Users\User\AppData\Local\Temp\ipykernel_78588\2186065473.py:3: RuntimeWarning: overflow encountered in exp
  np.exp(-0.008/gauss_t2s_vol)


array([[[0.99977055, 0.99977114, 0.99977125, ..., 0.8925378 ,
         0.87882073, 0.91057854],
        [0.99977119, 0.99977153, 0.99977154, ..., 1.20314532,
         1.17470295, 1.08173866],
        [0.99977128, 0.99977157, 0.99977206, ..., 0.76059126,
         1.18604519, 0.69485302],
        ...,
        [0.99953056, 0.99953071, 0.99952835, ..., 0.99975746,
         0.99975544, 0.99975725],
        [0.99953223, 0.99952696, 0.99953284, ..., 0.99975737,
         0.99975994, 0.99975919],
        [0.99952909, 0.99953284, 0.99952891, ..., 0.99975746,
         0.99975819, 0.99975755]],

       [[0.99977193, 0.99977118, 0.99977223, ..., 0.92954063,
         0.47896015, 0.92544232],
        [0.99977204, 0.99977139, 0.99977056, ..., 0.7543522 ,
         1.04446962, 2.07424807],
        [0.99977015, 0.99977226, 0.99977066, ..., 0.91326078,
         0.92713161, 1.32383754],
        ...,
        [0.999533  , 0.99953259, 0.99952786, ..., 0.99975757,
         0.99975771, 0.9997587 ],
        [0.9

In [12]:
phase

array([[[[ 0.54555659, -2.10104513],
         [-0.25586268, -1.87463326],
         [ 1.81236057,  0.58138185],
         ...,
         [ 2.41511543,  1.29715325],
         [-1.50609334, -2.18118493],
         [-2.70718134, -2.42937968]],

        [[ 2.93006893, -0.44753396],
         [-2.7905155 ,  1.39865182],
         [ 0.41268169, -1.08073682],
         ...,
         [-2.41869426,  3.0182868 ],
         [ 0.44388072,  0.13440928],
         [ 1.84095387,  2.97153088]],

        [[ 0.69432323, -3.10248199],
         [ 1.82428157, -0.5825592 ],
         [-0.55343919, -2.22800537],
         ...,
         [-1.31796368, -1.95778095],
         [ 2.69771924,  2.81084251],
         [-2.77893775, -1.33649317]],

        ...,

        [[ 2.6382018 ,  2.74016556],
         [-0.41060807, -0.88029617],
         [-0.36743729, -0.82903086],
         ...,
         [-0.97802009,  0.80209655],
         [-0.53316262,  1.3303648 ],
         [ 2.76826318, -2.21047462]],

        [[ 0.42369208,  0.11043526

In [9]:
# Finally we save the magnitude and the phase to analyze them
# Remember to change the name, if not it will overwrite it!
temp_mag = nib.Nifti1Image(magnitude, affine=data.affine)
temp_ph = nib.Nifti1Image(phase, affine=data.affine)
path_mag = os.path.join('output','realistic3_sim_mag.nii.gz')
path_ph = os.path.join('output','realistic3_sim_phase.nii.gz')
nib.save(temp_mag, path_mag)
nib.save(temp_ph, path_ph)

In [8]:
# Finally we save the magnitude and the phase to analyze them
# Remember to change the name, if not it will overwrite it!
# This version for gaussian distributed volumes
temp_mag_g = nib.Nifti1Image(gauss_magnitude, affine=data.affine)
temp_ph_g = nib.Nifti1Image(gauss_phase, affine=data.affine)
path_mag_g = os.path.join('output','gauss_sim_mag.nii.gz')
path_ph_g = os.path.join('output','gauss_sim_phase.nii.gz')
nib.save(temp_mag_g, path_mag_g)
nib.save(temp_ph_g, path_ph_g)


In [18]:
gauss_magnitude[3,40,100]

array([0., 0.])